In [71]:
import sys
sys.path.append("..")
import importlib
import experimenter
from experimenter.utils import text

importlib.reload(experimenter)
importlib.reload(text)

<module 'experimenter.utils.text' from '../experimenter/utils/text.py'>

In [187]:
# pair stance classification
import pandas as pd
from experimenter import evaluation
from experimenter.utils import modeling, training, data
importlib.reload(modeling)
importlib.reload(evaluation)
importlib.reload(training)
importlib.reload(data)

configs = {"epochs": 3,
           "experiment_output_path": "final_training.json", 
           "log_interval": 2, "model_path": "model.state", 
           "processor": {
                "module": "experimenter.utils.data",
                "class": "PairStanceProvider",
                "params":{"input_path": "/Users/jkhouja/workspace/repo/arabic_media/data/pairs/batch_0_to_15000_pairs_sep__score_10_ngrams_2_3_4_5_6.csv",
                          "seq_len": [[20,20],[1],[1]], "batch_size": 20, "splits": [.1, .1, .1, .7], "drop_last":"False",  "shuffle": "True", "vocab_path": "vocab.json"}},
           "model":{
                "module": "experimenter.utils.modeling",
                "class": "RNNPairModel",
                "params":{"embedding_dim": 8, "hidden_dim": 10, "num_classes": {"eval":1, "value":"config['processor']['params']['vocab_size']"}, "dropout": 0, "max_seq_len": {"eval": 1, "value": "config['processor']['params']['seq_len'][0]"}}},
           
            "evaluator":{
                "module": "experimenter.evaluation",
                "class": "ListEvaluator",
                "params": {"loss_f": [{"module": 'torch.nn', "class": 'CrossEntropyLoss', "params": {'reduction': 'none'}}]}},
           "optimizer":{
                "module": "torch.optim",
                "class": "Adam",
                "params":{
                    "params": {
                    "eval": 1,
                    "value": "config['model']['model'].parameters()"},
                'lr':  0.001}}
           }


trainer = training.BasicTrainer(configs)
trainer.train_model()




Will create train, dev, test(s) splits
Total params: 2245
Starting training:
Epoch: 0: Train loss (last batch): 4.161691665649414, validation loss: 81.64664459228516
Best model saved at: model.state
Epoch: 2: Train loss (last batch): 2.151963233947754, validation loss: 38.215728759765625
Best model saved at: model.state
tensor(38.0964, grad_fn=<DivBackward0>)


{'epochs': 3,
 'experiment_output_path': 'final_training.json',
 'log_interval': 2,
 'model_path': 'model.state',
 'processor': {'module': 'experimenter.utils.data',
  'class': 'PairStanceProvider',
  'params': {'input_path': '/Users/jkhouja/workspace/repo/arabic_media/data/pairs/batch_0_to_15000_pairs_sep__score_10_ngrams_2_3_4_5_6.csv',
   'seq_len': [[20, 20], [1], [1]],
   'batch_size': 20,
   'splits': [0.1, 0.1, 0.1, 0.7],
   'drop_last': 'False',
   'shuffle': 'True',
   'vocab_path': 'vocab.json',
   'vocab_size': 75}},
 'model': {'module': 'experimenter.utils.modeling',
  'class': 'RNNPairModel',
  'params': {'embedding_dim': 8,
   'hidden_dim': 10,
   'num_classes': 75,
   'dropout': 0,
   'max_seq_len': [20, 20]},
  'model': RNNPairModel(
    (emb): Embedding(75, 8, padding_idx=0)
    (lstm): LSTM(8, 10, batch_first=True)
    (linear): Linear(in_features=10, out_features=75, bias=True)
    (batch_norm): BatchNorm1d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stat

In [190]:
# Predict on unlabled data

limit = 10
unlabeled_data = [["عبر رجل", "قال رجل"]] * 7

pred = trainer.predict(unlabeled_data, decode=False)
pred

[[1, 1, 1, 1, 1, 1, 1]]

In [185]:
# Predict on unlabled data
eval_data = trainer.processor.get_data()[1]
limit = 10
unlabeled_data = [["عبر رجل", "قال رجل"]] * 7

pred = trainer.predict(unlabeled_data)

# Get entropy, Select data

entr = get_ent(pred_entr)

selected_idx = np.argmax(entr, limit)


selected_data = unlabeled_data[selected_idx]

# Get labeles
selected_data = [[["قال رجل", "قال رجل"],[['agree']],[1]]]*2

#Retrain
this_data = trainer.processor(selected_data, data_type='full', list_input= True, as_batches=True)
trainer.train_model([this_data, eval_data])

Starting training:
Epoch: 2: Train loss (last batch): 0.7104984521865845, validation loss: 28.14354133605957
Best model saved at: model.state


{'epochs': 3,
 'experiment_output_path': 'final_training.json',
 'log_interval': 2,
 'model_path': 'model.state',
 'processor': {'module': 'experimenter.utils.data',
  'class': 'PairStanceProvider',
  'params': {'input_path': '/Users/jkhouja/workspace/repo/arabic_media/data/pairs/batch_0_to_15000_pairs_sep__score_10_ngrams_2_3_4_5_6.csv',
   'seq_len': [[20, 20], [1], [1]],
   'batch_size': 20,
   'splits': [0.1, 0.1, 0.1, 0.7],
   'drop_last': 'False',
   'shuffle': 'True',
   'vocab_path': 'vocab.json',
   'vocab_size': 75}},
 'model': {'module': 'experimenter.utils.modeling',
  'class': 'RNNPairModel',
  'params': {'embedding_dim': 8,
   'hidden_dim': 10,
   'num_classes': 75,
   'dropout': 0,
   'max_seq_len': [20, 20]},
  'model': RNNPairModel(
    (emb): Embedding(75, 8, padding_idx=0)
    (lstm): LSTM(8, 10, batch_first=True)
    (linear): Linear(in_features=10, out_features=75, bias=True)
    (batch_norm): BatchNorm1d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stat

In [249]:
import numpy as np
import torch
importlib.reload(data)

class mydict(dict):
    
    def __init__(self, *args, **kwargs):
        super(mydict, self).__init__(*args, **kwargs)
        self.len = 6
    pass

    

data_as_dict = mydict({'in':np.zeros(10), 'out':np.ones(10)})
data_as_dict = data.DictDataset(data_as_dict)
d = torch.utils.data.DataLoader(dataset=data_as_dict, batch_size=2, drop_last=False, shuffle=False)
for b in d:
    print(b)

{}
{}
{}
{}
{}


In [220]:
data_as_dict = mydict({'in':np.zeros(10), 'out':np.ones(10)})
s = data_as_dict.__class__()
assert isinstance(s, dict)
s

{}

In [212]:
s = mydict({'in':np.zeros(10), 'out':np.ones(10)})
__main__.mydict()

NameError: name '__main__' is not defined

In [110]:
t = trainer.evaluate([[["قال رجل", "قال رجل"],[['agree']],[1]],[["قال رجل", "قال رجل"],[['agreasdfde']],[1]]])
t

tensor(6.3336, grad_fn=<DivBackward0>)

In [142]:
trainer.processor([["قال رجل", "قال رجل"],[['agree']],[1]], data_type='full', as_batches=True)

In [186]:
t = trainer.predict([["عبر رجل", "قال رجل"]] * 7)
t

['agree', 'agree', 'agree', 'agree', 'agree', 'agree', 'agree']

In [62]:
t = trainer.predict([["عبر رجل", "قال رجل"]] * 7)
#len(t[0][0])
res = []
for b in t:
    try:
        res.extend([s.argmax(dim=1).tolist() for s in b])
    except IndexError as e:
        # batch_size = 1 or last batch
        res.extend([[s.argmax().tolist() for s in b]])
res

AttributeError: 'str' object has no attribute 'argmax'

In [46]:
t = trainer.predict([["hi there", "man"]])
t

RuntimeError: index out of range: Tried to access index 75 out of table with 74 rows. at ../aten/src/TH/generic/THTensorEvenMoreMath.cpp:418

In [ ]:
trainer.processor([["hi there", "man"]] * 4, list_input= True, as_batches=True)

In [ ]:
s = trainer.processor([["hi_there man_", "test this"]] * 3, list_input=True)
s

In [ ]:
s = ['hi', 'by']
tuple(s)


In [252]:
!python ../run.py --config_file "../local/pairs.json"

Will create train, dev, test(s) splits
Total params: 2245
Starting training:
Epoch: 0: Train loss (last batch): 4.127970218658447, validation loss: 82.26912689208984
Best model saved at: model.state
Epoch: 2: Train loss (last batch): 1.525651454925537, validation loss: 31.06138038635254
Best model saved at: model.state
tensor(31.1143, grad_fn=<DivBackward0>)
Traceback (most recent call last):
  File "../run.py", line 23, in <module>
    config = train_model()
NameError: name 'train_model' is not defined


In [ ]:
def process(s):
    tokenizer = text.tokenizer(sep='')
    cleaner = text.clean_text('_')
    #cleaner2 = text.clean_text('h')
    enc = text.encoder(update_vocab=True, no_special_chars=False)
    pipeline = text.chainer(funcs=[cleaner, tokenizer, enc])
    res = []
    for inp in s:
        proc = pipeline(inp, list_input=False)
        res.append([[proc, proc], [[2],[1]], [1, 100]])
    return res, enc #need to fix returned objects it's a mess

In [ ]:

np.argmax(model(iter(d1).next())[0][0].detach().numpy(), axis=1)

In [ ]:
model(iter(d1).next())[0]

In [ ]:
s = a(["hi_there man_", "how are you?"], list_input=True)

In [ ]:
def test_chainer():
    tokenizer = text.tokenizer(sep=' ')
    enc = text.encoder(update_vocab=True)
    chain = text.chainer(funcs=[tokenizer, enc, enc.decode, tokenizer.detokenize])
    
    inp = "مرحبا هنا"
    assert inp == chain(inp, list_input=False)
    
    inp = ["hi_there man_", "how are you?"]
    assert inp == chain(inp, list_input=True)

test_chainer()

In [ ]:
text.list_applyer
cleaner = text.clean_text('_')

cleaner("hi")

In [ ]:
class a:
    #self.b = "hi"
    def __init__(self):
        self.b = "hi"
    def test(self, text=b):
        print(text)
tryme = a()
tryme.test()